In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession.builder.master("local").appName("spark-sql-table").getOrCreate()

24/08/15 22:05:31 WARN Utils: Your hostname, MZC01-HYUCKSANGCHO.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
24/08/15 22:05:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/15 22:05:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
dfCalendar = spark.read.load("airbnb_data/airbnb_calendar.csv",
                     format="csv", inferSchema=True, header=True,
                     quote='"', escape='"', sep=',', multiline=True)


dfListing = spark.read.load("airbnb_data/airbnb_listings.csv",
                     format="csv", inferSchema=True, header=True,
                     quote='"', escape='"', sep=',', multiline=True)

dfCalendar = dfCalendar.cache()
dfListing = dfListing.cache()

24/08/15 22:05:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
dfCalendar.printSchema()

root
 |-- listing_id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- available: string (nullable = true)
 |-- price: string (nullable = true)
 |-- adjusted_price: string (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- maximum_nights: integer (nullable = true)



In [4]:
dfCalendar.show()

+----------+----------+---------+-------+--------------+--------------+--------------+
|listing_id|      date|available|  price|adjusted_price|minimum_nights|maximum_nights|
+----------+----------+---------+-------+--------------+--------------+--------------+
|   1153002|2019-11-29|        f|$145.00|       $145.00|            30|            60|
|   3138055|2019-11-29|        f|$130.00|       $130.00|             2|          1125|
|   3138055|2019-11-30|        f|$130.00|       $130.00|             2|          1125|
|   3138055|2019-12-01|        t|$130.00|       $130.00|             2|          1125|
|   3138055|2019-12-02|        f|$130.00|       $130.00|             2|          1125|
|   3138055|2019-12-03|        f|$130.00|       $130.00|             2|          1125|
|   3138055|2019-12-04|        f|$130.00|       $130.00|             2|          1125|
|   3138055|2019-12-05|        f|$130.00|       $130.00|             2|          1125|
|   3138055|2019-12-06|        f|$130.00|  

In [5]:
dfListing.printSchema()

root
 |-- id: integer (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: date (nullable = true)
 |-- name: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- space: string (nullable = true)
 |-- description: string (nullable = true)
 |-- experiences_offered: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- notes: string (nullable = true)
 |-- transit: string (nullable = true)
 |-- access: string (nullable = true)
 |-- interaction: string (nullable = true)
 |-- house_rules: string (nullable = true)
 |-- thumbnail_url: string (nullable = true)
 |-- medium_url: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- xl_picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_location: string (nullable = t

In [6]:
dfListing\
    .select("id", "listing_url", "name", "description", "property_type", "city", "review_scores_rating", "price")\
    .show()

+------+--------------------+--------------------+--------------------+-------------+---------+--------------------+-------+
|    id|         listing_url|                name|         description|property_type|     city|review_scores_rating|  price|
+------+--------------------+--------------------+--------------------+-------------+---------+--------------------+-------+
|   360|https://www.airbn...|LoHi Secret garde...|Come enjoy our oa...|   Guesthouse|   Denver|                 100|$140.00|
|   590|https://www.airbn...|Comfortable  - an...|Large guest room ...|        House|   Denver|                  97| $61.00|
|   592|https://www.airbn...|             private|This room is in t...|        House|   Denver|                  97| $44.00|
|  1940|https://www.airbn...|Baker Studio Clos...|Great place for a...|   Guesthouse|   Denver|                  99| $95.00|
|  2086|https://www.airbn...|  Garden Level Condo|A furnished, gard...|    Apartment|   Denver|                  96| $76.00|


In [7]:
# createTempView : Spark Session 별로 분리
# createGlobalTempVeiw : 서로 다른 Session간 공유
dfListing.createOrReplaceTempView("LISTING_META")
dfCalendar.createOrReplaceTempView("LISTING_CALENDAR")

In [11]:
propertyTypeHouse = 'House'

dfListingHouseAvailable = spark.sql(f"""
WITH LISTING_HOUSE AS (
    SELECT 
        id as listing_id,
        listing_url,
        name
    FROM 
        LISTING_META
    WHERE property_type = '{propertyTypeHouse}'
    )
SELECT
    LISTING_CALENDAR.listing_id,
    listing_url, 
    name as listing_name,
    date,
    available,
    price
FROM LISTING_CALENDAR, LISTING_HOUSE
WHERE LISTING_CALENDAR.listing_id = LISTING_HOUSE.listing_id
AND LISTING_CALENDAR.available = 't'

""")
dfListingHouseAvailable.show()

+----------+--------------------+--------------------+----------+---------+-------+
|listing_id|         listing_url|        listing_name|      date|available|  price|
+----------+--------------------+--------------------+----------+---------+-------+
|   3138055|https://www.airbn...|Classic Home on C...|2019-12-01|        t|$130.00|
|   3138055|https://www.airbn...|Classic Home on C...|2019-12-13|        t|$130.00|
|   3138055|https://www.airbn...|Classic Home on C...|2019-12-14|        t|$130.00|
|   3138055|https://www.airbn...|Classic Home on C...|2019-12-15|        t|$130.00|
|   3138055|https://www.airbn...|Classic Home on C...|2019-12-21|        t|$130.00|
|   3138055|https://www.airbn...|Classic Home on C...|2019-12-22|        t|$130.00|
|   3138055|https://www.airbn...|Classic Home on C...|2019-12-26|        t|$130.00|
|   3138055|https://www.airbn...|Classic Home on C...|2019-12-27|        t|$130.00|
|   3138055|https://www.airbn...|Classic Home on C...|2019-12-28|        t|$

In [12]:
dfListingAvailability = spark.sql("""
WITH LISTING_GROUPED AS (
    SELECT 
        listing_id, 
        count(CASE WHEN available = 't' THEN 1 END) as count_date_available,
        count(CASE WHEN available <> 't' THEN 1 END) as count_date_unavailable
        
    FROM LISTING_CALENDAR
    GROUP BY listing_id
)

SELECT 
    LISTING_META.id as listing_id,
    LISTING_META.listing_url as listing_url,
    LISTING_META.name as listing_name,
    LISTING_META.review_scores_rating as review_scores_rating,
    LISTING_META.price as listing_price_basic,
    coalesce(LISTING_GROUPED.count_date_available, 0) as count_date_available,
    coalesce(LISTING_GROUPED.count_date_unavailable, 0) as count_date_unavailable

FROM LISTING_META
LEFT JOIN LISTING_GROUPED 
    ON LISTING_META.id = LISTING_GROUPED.listing_id
""")
dfListingAvailability.show()

+----------+--------------------+--------------------+--------------------+-------------------+--------------------+----------------------+
|listing_id|         listing_url|        listing_name|review_scores_rating|listing_price_basic|count_date_available|count_date_unavailable|
+----------+--------------------+--------------------+--------------------+-------------------+--------------------+----------------------+
|    192430|https://www.airbn...|TREETOP VIEW ROOM...|                  94|             $32.00|                  61|                   304|
|       360|https://www.airbn...|LoHi Secret garde...|                 100|            $140.00|                 325|                    40|
|     98014|https://www.airbn...|Beautiful single ...|                  98|             $60.00|                 211|                   154|
|       590|https://www.airbn...|Comfortable  - an...|                  97|             $61.00|                 116|                   249|
|     74125|https://

In [13]:
dfListingAvailability.orderBy(desc(col("count_date_unavailable"))).show()

+----------+--------------------+--------------------+--------------------+-------------------+--------------------+----------------------+
|listing_id|         listing_url|        listing_name|review_scores_rating|listing_price_basic|count_date_available|count_date_unavailable|
+----------+--------------------+--------------------+--------------------+-------------------+--------------------+----------------------+
|  20425070|https://www.airbn...|Location, locatio...|                  91|            $150.00|                   0|                   366|
|  39309393|https://www.airbn...|Guest suite Denve...|                null|             $80.00|                   0|                   366|
|  37912396|https://www.airbn...|Happy Travelers W...|                 100|             $75.00|                   0|                   366|
|  14154970|https://www.airbn...|Best Location! Gr...|                  99|            $229.00|                   0|                   366|
|  27332948|https://

In [15]:
spark.sql("""
CACHE TABLE LISTING_STAT AS (
    SELECT *
    FROM LISTING_META
    WHERE review_scores_rating IS NOT NULL
)
""")

spark.sql("SHOW TABLES").show()

+---------+----------------+-----------+
|namespace|       tableName|isTemporary|
+---------+----------------+-----------+
|         |listing_calendar|       true|
|         |    listing_meta|       true|
|         |    listing_stat|       true|
+---------+----------------+-----------+

